In [1]:
!pip install pandas numpy scikit-learn gensim bertopic google-play-scraper sentence-transformers
!git clone https://github.com/rwalk/gsdmm.git
%cd gsdmm
!pip install .
%cd ..


fatal: destination path 'gsdmm' already exists and is not an empty directory.
/content/gsdmm
Processing /content/gsdmm
  Preparing metadata (setup.py) ... done
  Created wheel for gsdmm: filename=gsdmm-0.1-py3-none-any.whl size=4587 sha256=dd0e3a2a850ab8a52deba61925eb5f3f25dd0f0aecf56480e36f936e778b0ae4
  Stored in directory: /tmp/pip-ephem-wheel-cache-o7dy_g6e/wheels/9b/34/a3/5925779cb6fad0c53d3ad0d6a10317967ae26ae21de05f519e
Successfully built gsdmm
  Attempting uninstall: gsdmm
    Found existing installation: gsdmm 0.1
    Uninstalling gsdmm-0.1:
      Successfully uninstalled gsdmm-0.1
/content


In [ ]:
# import pandas as pd

# # Load the provided CSV
# df = pd.read_csv("/content/googleplaystore_user_reviews.csv")

# # Try to detect review text column
# possible_cols = ['review', 'content', 'body', 'comment', 'text', 'Translated_Review']
# found_col = None
# for col in df.columns:
#     if col.lower() in [p.lower() for p in possible_cols]:
#         found_col = col
#         break

# if found_col is None:
#     raise ValueError("No review text column found. Please rename manually.")
# else:
#     df.rename(columns={found_col: 'review'}, inplace=True)

# # Keep only 'review' column
# df = df[['review']].dropna().reset_index(drop=True)

# print("Dataset shape:", df.shape)
# df.head(5)


In [5]:
# Core libraries
import pandas as pd
import numpy as np
import re

# NLP & clustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Transformers
from transformers import pipeline

# BERTopic & GSDMM
# If not installed, uncomment these:
!pip install bertopic
!pip install gensim
!pip install sentence-transformers

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


In [8]:
# Load your dataset
df = pd.read_csv("googleplaystore_user_reviews.csv")  # Adjust path if needed

# Basic cleaning function
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', str(text))  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text.lower())  # Remove punctuation and lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text.strip()

# Apply cleaning to 'content' column
df['cleaned'] = df['content'].apply(clean_text)

# Drop rows with empty or null cleaned text
df = df[df['cleaned'].str.strip().astype(bool)]
df.reset_index(drop=True, inplace=True)

# Preview
df[['content', 'cleaned']].head()


,content,cleaned
0,I cannot open the app anymore,i cannot open the app anymore
1,I have been begging for a refund from this app...,i have been begging for a refund from this app...
2,Very costly for the premium version (approx In...,very costly for the premium version approx ind...
3,"Used to keep me organized, but all the 2020 UP...",used to keep me organized but all the updates...
4,Dan Birthday Oct 28,dan birthday oct


In [ ]:
df.shape

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF
tfidf = TfidfVectorizer(max_df=0.9, min_df=10, stop_words='english')

# Fit and transform the cleaned reviews
X_tfidf = tfidf.fit_transform(df['cleaned'])

# Print shape for sanity check
print(f"TF-IDF matrix shape: {X_tfidf.shape}")


TF-IDF matrix shape: (12463, 1744)


In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Initialize and fit KMeans
kmeans = KMeans(n_clusters=10, random_state=42)
df['cluster'] = kmeans.fit_predict(X_tfidf)

# Evaluate clustering quality
score = silhouette_score(X_tfidf, df['cluster'])
print(f"Silhouette Score: {score:.3f}")


Silhouette Score: 0.027


In [11]:
# Get feature names from TF-IDF
terms = tfidf.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

# Print top 10 terms per cluster
for i in range(10):
    top_terms = [terms[ind] for ind in order_centroids[i, :10]]
    print(f"Cluster {i}: {', '.join(top_terms)}")


Cluster 0: students, superb, app, good, useful, like, nice, dont, use, amazing
Cluster 1: tasks, time, task, app, list, day, add, like, set, just
Cluster 2: app, love, like, nice, good, version, really, work, best, premium
Cluster 3: working, app, stopped, properly, notifications, widget, good, really, phone, notification
Cluster 4: good, app, far, pretty, apps, use, really, application, quite, job
Cluster 5: great, app, nice, tool, love, really, far, day, life, works
Cluster 6: excellent, app, easy, planner, helpful, use, features, calendar, application, daily
Cluster 7: habits, track, app, good, habit, free, like, great, tracking, new
Cluster 8: easy, use, simple, app, good, helpful, useful, great, love, nice
Cluster 9: calendar, google, app, sync, events, use, like, dont, ads, tasks


In [12]:
# Sample 3 reviews from each of the first 5 clusters
for i in range(5):
    print(f"\n📌 Cluster {i} Samples:")
    samples = df[df['cluster'] == i]['content'].sample(3, random_state=42).to_list()
    for idx, review in enumerate(samples, 1):
        print(f"{idx}. {review}")



📌 Cluster 0 Samples:
1. Fun for university students The only problem is they don't order the tasks by date, do you have to be careful not to miss homework. Now that i think about it, it's actually a big problem for students using the app. It is fun, overall, and I do homework to keep my avatars alive. Think Tamagotchi
2. It is a superb app I like this app very much
3. include monthly to do list. which helps students to create timetable.

📌 Cluster 1 Samples:
1. Installed this app, added the tasks, logged in using google in the same order. But after login the tasks added no more found. This is definitely unacceptable.
2. Good app and I like the setups that are already prepared for when u log in. My problem with the app is that it's too overwhelming. Too many things to manage and for my easily distracted and anxious brain it's too much. As constructive that can actually be applied--please make it so the tasks actually leave the list ocne they are completed. Some are infinite and its dif

In [13]:
from transformers import pipeline

# Load Flan-T5 model for text2text generation
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Example vague review
vague_review = "App crashes often and doesn't load properly"
prompt = f"Respond to user complaint: '{vague_review}'"

# Generate response
response = generator(prompt, max_length=50, do_sample=True)
print("Generated Response:", response[0]['generated_text'])



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Response: It crashes often and doesn't load properly


In [14]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Load sentence transformer model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings from cleaned reviews
embeddings = embedding_model.encode(df['cleaned'].tolist(), show_progress_bar=True)

# Fit BERTopic model
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(df['cleaned'].tolist(), embeddings)

# Assign topics to DataFrame
df['bertopic_cluster'] = topics

# Show top 10 topics
topic_model.get_topic_info().head(10)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/390 [00:00<?, ?it/s]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4246,-1_to_and_it_the,"[to, and, it, the, for, my, tasks, app, is, you]",[i really like the application so far great in...
1,0,622,0_calendar_google_calendars_events,"[calendar, google, calendars, events, sync, ca...","[great calendar, good calendar app, not sync w..."
2,1,270,1_habit_habits_tracker_track,"[habit, habits, tracker, track, tracking, only...","[only habit you can update, you can only add ..."
3,2,246,2_trees_tree_coins_forest,"[trees, tree, coins, forest, pro, plant, speci...","[i like trees, i loved this app a lot however ..."
4,3,236,3_life_rpg_game_fun,"[life, rpg, game, fun, goals, and, you, more, ...",[it sounds crazy but this app has basically ch...
5,4,171,4_good_swell_rathergood_goodd,"[good, swell, rathergood, goodd, yes, experime...","[good, good, good]"
6,5,170,5_ads_full_ad_screen,"[ads, full, ad, screen, annoying, video, intru...",[used to be without ads now the ads are annoyi...
7,6,161,6_planner_paper_digital_planners,"[planner, paper, digital, planners, agenda, ar...","[good planner apps, great planner for me, best..."
8,7,143,7_sync_devices_data_syncing,"[sync, devices, data, syncing, phone, premium,...",[could be a great app except that like another...
9,8,142,8_easy_simple_very_helpful,"[easy, simple, very, helpful, app, use, this, ...","[this is a great app easy to use, very helpful..."


In [15]:
# Install GSDMM if needed
# !pip install gsdmm

from gsdmm import MovieGroupProcess
from sklearn.feature_extraction.text import CountVectorizer

# Tokenize reviews for GSDMM
df['tokens'] = df['cleaned'].apply(lambda x: x.split())

# Initialize GSDMM
mgp = MovieGroupProcess(K=20, alpha=0.1, beta=0.3, n_iters=30)

# Fit model
vocab = set(word for doc in df['tokens'] for word in doc)
n_terms = len(vocab)
doc_list = df['tokens'].tolist()

mgp.fit(doc_list, n_terms)

# Assign cluster labels
df['gsdmm_cluster'] = [mgp.choose_best_label(doc)[0] for doc in doc_list]

# Show top 10 clusters
from collections import Counter
print("Top GSDMM clusters:", Counter(df['gsdmm_cluster']).most_common(10))


In stage 0: transferred 11579 clusters with 20 clusters populated
In stage 1: transferred 7434 clusters with 20 clusters populated
In stage 2: transferred 2796 clusters with 20 clusters populated
In stage 3: transferred 1858 clusters with 16 clusters populated
In stage 4: transferred 1708 clusters with 14 clusters populated
In stage 5: transferred 1636 clusters with 12 clusters populated
In stage 6: transferred 1695 clusters with 10 clusters populated
In stage 7: transferred 1672 clusters with 11 clusters populated
In stage 8: transferred 1700 clusters with 9 clusters populated
In stage 9: transferred 1634 clusters with 10 clusters populated
In stage 10: transferred 1606 clusters with 11 clusters populated
In stage 11: transferred 1752 clusters with 10 clusters populated
In stage 12: transferred 1758 clusters with 9 clusters populated
In stage 13: transferred 1661 clusters with 8 clusters populated
In stage 14: transferred 1607 clusters with 9 clusters populated
In stage 15: transferre

In [16]:
from collections import Counter

# TF-IDF + KMeans
print("\n🔹 TF-IDF + KMeans:")
print("Top clusters:", Counter(df['cluster']).most_common(5))

# BERTopic
print("\n🔹 BERTopic:")
print("Top topics:", Counter(df['bertopic_cluster']).most_common(5))

# GSDMM
print("\n🔹 GSDMM:")
print("Top clusters:", Counter(df['gsdmm_cluster']).most_common(5))



🔹 TF-IDF + KMeans:
Top clusters: [(2, 7414), (1, 2421), (9, 660), (5, 450), (8, 430)]

🔹 BERTopic:
Top topics: [(-1, 4246), (0, 622), (1, 270), (2, 246), (3, 236)]

🔹 GSDMM:
Top clusters: [(9, 7574), (11, 4449), (19, 114), (4, 103), (12, 82)]


In [17]:
# Filter vague reviews: short length or low score
vague_df = df[(df['score'] <= 2) | (df['cleaned'].str.len() < 40)].copy()

# Load Flan-T5 pipeline
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Generate responses for first 5 vague reviews
for i, row in vague_df.head(5).iterrows():
    prompt = f"Respond to user complaint: '{row['content']}'"
    response = generator(prompt, max_length=50, do_sample=True)
    print(f"\n🗣️ Original: {row['content']}")
    print(f"🤖 Response: {response[0]['generated_text']}")


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🗣️ Original: I cannot open the app anymore
🤖 Response: I've been using the app for over a year now and I've been using it for about a year now and I can't find anything on the app. I haven't been able to find anything on the app and I can't find anything on the app. I've been using the app for over a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I've been using the app for about a year now and I can't find anything on the app. I'


Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🗣️ Original: I have been begging for a refund from this app for over a month and nobody is replying me
🤖 Response: 'I have been begging for a refund from this app for over a month and nobody is replying me'

🗣️ Original: Very costly for the premium version (approx Indian Rupees 910 per year). Better to download the premium version of this app from apkmos website and use it. Microsoft to do list app is far more better.
🤖 Response: Very costly for the premium version (approx Indian Rupees 910 per year). Better to download the premium version of this app from apkmos website and use it. Microsoft to do list app is far more better


Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🗣️ Original: Used to keep me organized, but all the 2020 UPDATES have made a mess of things !!! Y cudn't u leave well enuf alone ??? Guess ur techies feel the need to keep making changes to justify continuing to collect their salary !!! 🤤🤤🤤
🤖 Response: : 'It makes me feel like I am wasting my time .'                                                                                                                       

🗣️ Original: Dan Birthday Oct 28
🤖 Response: Dan's birthday is on Oct 28. I don't know if it's a birthday or if it's a birthday or if it's just a birthday.


In [18]:
# Select relevant columns for export
export_cols = [
    'reviewId', 'userName', 'content', 'score',
    'cleaned', 'cluster', 'bertopic_cluster', 'gsdmm_cluster'
]

# Export to CSV
df[export_cols].to_csv("revai_clustered_reviews.csv", index=False)

print("✅ Export complete: revai_clustered_reviews.csv")


✅ Export complete: revai_clustered_reviews.csv


In [ ]:
from tqdm import tqdm

# Reload Flan-T5 pipeline if needed
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Filter vague reviews (short or low score)
vague_df = df[(df['score'] <= 2) | (df['cleaned'].str.len() < 40)].copy()

# Initialize response column
vague_df['flan_response'] = ""

# Generate responses
for i, row in tqdm(vague_df.iterrows(), total=len(vague_df)):
    prompt = f"Respond to user complaint: '{row['content']}'"
    try:
        response = generator(prompt, max_length=50, do_sample=True)
        vague_df.at[i, 'flan_response'] = response[0]['generated_text']
    except Exception as e:
        vague_df.at[i, 'flan_response'] = "Error generating response"

# Merge responses back into main DataFrame
df = df.merge(vague_df[['reviewId', 'flan_response']], on='reviewId', how='left')

# Export final enriched dataset
df.to_csv("revai_clustered_with_responses.csv", index=False)
print("✅ Export complete: revai_clustered_with_responses.csv")


Device set to use cpu
  1%|▏         | 99/7291 [30:39<24:14:59, 12.14s/it]Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np

# Train Word2Vec on tokenized reviews
tokenized_reviews = df['cleaned'].apply(lambda x: x.split())
w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=5, workers=4)

# Create sentence embeddings by averaging word vectors
def get_sentence_vector(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

df['w2v_vector'] = tokenized_reviews.apply(get_sentence_vector)

# Stack vectors into matrix
X_w2v = np.vstack(df['w2v_vector'].values)

# Cluster using KMeans
w2v_kmeans = KMeans(n_clusters=10, random_state=42)
df['w2v_cluster'] = w2v_kmeans.fit_predict(X_w2v)

print("✅ Word2Vec clustering complete. Sample clusters:", df['w2v_cluster'].value_counts().head())


In [ ]:
from transformers import pipeline

# Load zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="distilbert-base-uncased")

# Define labels
labels = ["vague", "detailed"]

# Classify first 10 reviews
df['distilbert_label'] = ""

for i, row in df.head(10).iterrows():
    result = classifier(row['content'], candidate_labels=labels)
    df.at[i, 'distilbert_label'] = result['labels'][0]

print(df[['content', 'distilbert_label']].head(10))


In [ ]:
from tqdm import tqdm

# Reload Flan-T5 pipeline if needed
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Filter reviews labeled as vague
vague_reviews = df[df['distilbert_label'] == "vague"].copy()
vague_reviews['flan_response'] = ""

# Generate responses
for i, row in tqdm(vague_reviews.iterrows(), total=len(vague_reviews)):
    prompt = f"Respond to user complaint: '{row['content']}'"
    try:
        response = generator(prompt, max_length=50, do_sample=True)
        vague_reviews.at[i, 'flan_response'] = response[0]['generated_text']
    except Exception:
        vague_reviews.at[i, 'flan_response'] = "Error generating response"

# Merge responses back into main DataFrame
df = df.merge(vague_reviews[['reviewId', 'flan_response']], on='reviewId', how='left')

print("✅ Flan-T5 responses generated for vague reviews only.")


In [ ]:
# Export enriched dataset with all clusters and responses
df.to_csv("revai_full_pipeline_output.csv", index=False)
print("📦 Final export complete: revai_full_pipeline_output.csv")
